You may find this series of notebooks at https://github.com/databricks-industry-solutions/oncology. For more information about this solution accelerator, visit https://www.databricks.com/solutions/accelerators/nlp-oncology.

# Abstracting Real World Data from Oncology Notes: Data Analysis

In the previous notebook (`./00-entity-extraction`) we used SparkNLP's pipelines to extract hightly specialized oncological entities from unstructured notes and stored the resulting tabular data in our delta lake.

In this notebook we analyze these data to answer questions such as:
What are the most common cancer subtypes? What are the most common symptoms and how are these symptoms associated with each cancer subtype? which indications have the highest risk factor? etc.

In [0]:
!pip install mlflow

  Using cached mlflow-2.1.1-py3-none-any.whl (16.7 MB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached docker-6.0.1-py3-none-any.whl (147 kB)
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached shap-0.41.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (572 kB)
  Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached SQLAlchemy-1.4.46-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached alembic-1.9.4-py3-none-any.whl (210 kB)
  Using cached Flask-2.2.3-py3-none-any.whl (101 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
  Using 

In [0]:
import mlflow
import numpy as np
import pandas as pd
from pyspark.sql import functions as F

In [0]:
mlflow.set_tracking_uri('databricks')

**Important Note! After running the cell above, please detach & re-attach to the cluster and continue.**

In [0]:
import mlflow
import numpy as np
import pandas as pd
from pyspark.sql import functions as F

In [0]:
%run ./16-onco_config

In [0]:
ade_demo_util=SolAccUtil('onc-lh',data_path='/FileStore/HLS/nlp/data/')
ade_demo_util.print_info()
delta_path=ade_demo_util.settings['delta_path']
# delta_path='/FileStore/HLS/nlp/delta/jsl/'

base_path = /home/meryem@johnsnowlabs.com/health-lakehouse delta_path = /home/meryem@johnsnowlabs.com/health-lakehouse/onc-lh/delta data_path = /FileStore/HLS/nlp/data/ experiment_name = /Users/meryem@johnsnowlabs.com/onc-lh experiment_id = 3779411755525061 artifact_location = dbfs:/databricks/mlflow-tracking/3779411755525061 tags = {'mlflow.ownerId': '5883233705899643', 'mlflow.experiment.sourceName': '/Users/meryem@johnsnowlabs.com/onc-lh', 'mlflow.ownerEmail': 'meryem@johnsnowlabs.com', 'mlflow.experimentType': 'MLFLOW_EXPERIMENT'}

let's take a look at the raw text dataset

In [0]:
df=spark.read.load(f'{delta_path}/bronze/mt-oc-notes')
display(df)

path text dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt Sample Type / Medical Specialty:
Hematology - Oncology
Sample Name:
Discharge Summary - Mesothelioma - 1
Description:
Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
(Medical Transcription Sample Report)
PRINCIPAL DIAGNOSIS:
Mesothelioma.
SECONDARY DIAGNOSES:
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
PROCEDURES
1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.
2. On August 20, 2007, thoracentesis.
3. On August 31, 2007, Port-A-Cath placement.
HISTORY AND PHYSICAL:
The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.
PAST MEDICAL HISTORY
1. Pericardectomy.
2. Pericarditis.
2. Atrial fibrillation.
4. RNCA with intracranial thrombolytic treatment.
5 PTA of MCA.
6. Mesenteric venous thrombosis.
7. Pericardial window.
8. Cholecystectomy.
9. Left thoracentesis.
FAMILY HISTORY:
No family history of coronary artery disease, CVA, diabetes, CHF or MI. The patient has one family member, a sister, with history of cancer.
SOCIAL HISTORY:
She is married. Employed with the US Post Office. She is a mother of three. Denies tobacco, alcohol or illicit drug use.
MEDICATIONS
1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.
2. Amiodarone 100 mg p.o. daily.
REVIEW OF SYSTEMS:
Complete review of systems negative except as in pulmonary as noted above. The patient also reports occasional numbness and tingling of her left arm.
PHYSICAL EXAMINATION
VITAL SIGNS: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%.
GENERAL: Positive nonproductive cough and pain with coughing.
HEENT: Pupils are equal and reactive to light and accommodation. Tympanic membranes are clear.
NECK: Supple. No lymphadenopathy. No masses.
RESPIRATORY: Pleural friction rub is noted.
GI: Soft, nondistended, and nontender. Positive bowel sounds. No organomegaly.
EXTREMITIES: No edema, no clubbing, no cyanosis, no tenderness. Full range of motion. Normal pulses in all extremities.
SKIN: No breakdown or lesions. No ulcers.
NEUROLOGIC: Grossly intact. No focal deficits. Awake, alert, and oriented to person, place, and time.
LABORATORY DATA:
Labs are pending.
HOSPITAL COURSE:
The patient was admitted for a right-sided pleural effusion for thoracentesis on Monday by Dr. X. Her Coumadin was placed on hold. A repeat echocardiogram was checked. She was started on prophylaxis for DVT with Lovenox 40 mg subcutaneously. Her history dated back to March 2005 when she first sought medical attention for evidence of pericarditis, which was treated with pericardial window in an outside hospital, at that time she was also found to have mesenteric pain and thrombosis, is now anticoagulated. Her pericardial fluid was accumulated and she was seen by Dr. Y. At that time, she was recommended for pericardectomy, which was performed by Dr. Z. Review of her CT scan from March 2006 prior to her pericardectomy, already shows bilateral plural effusions. The patient improved clinically after the pericardectomy with resolution of her symptoms. Recently, she was readmitted to the hospital with chest pain and found to have bilateral pleural effusion, the right greater than the left. CT of the chest also revealed a large mediastinal lymph node. We reviewed the pathology obtained from the pericardectomy in March 2006, which was diagnostic of mesothelioma. At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with f

# 1. ICD-10 Codes and HCC Status

Now we load the `icd10` delta tables

In [0]:
icd10_hcc_df=spark.read.load(f'{delta_path}/silver/icd10-hcc-df')
icd10_hcc_df.createOrReplaceTempView('icd10HccView')

best_icd_mapped_df=spark.read.load(f'{delta_path}/gold/best-icd-mapped')
best_icd_mapped_df.createOrReplaceTempView('bestIcdMappedView')
best_icd_mapped_pdf=best_icd_mapped_df.toPandas()

In [0]:
%sql
select * from icd10HccView
limit 10

path,final_chunk,entity,icd10_code,confidence,all_codes,resolutions,hcc_status,hcc_score,billable,icd_codes_names,icd_code_billable
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Mesothelioma,Oncological,C45,0.9986,"List(C45, C450, C459, C452, C457, C451, G731, C439, D165, C717, C649, C710, D352, C9000, C900)","List(mesothelioma [Mesothelioma], mesothelioma of pleura [Mesothelioma of pleura], mesothelioma, unspecified [Mesothelioma, unspecified], mesothelioma of pericardium [Mesothelioma of pericardium], mesothelioma of other sites [Mesothelioma of other sites], mesothelioma of mesentery [Mesothelioma of peritoneum], cancer, mesothelioma [Lambert-Eaton syndrome in neoplastic disease], amelanotic melanoma [Malignant melanoma of skin, unspecified], ameloblastoma of mandible [Benign neoplasm of lower jaw bone], glioma of brainstem [Malignant neoplasm of brain stem], nephroblastoma [Malignant neoplasm of unspecified kidney, except renal pelvis], glioblastoma multiforme, cerebrum [Malignant neoplasm of cerebrum, except lobes and ventricles], pituitary microadenoma [Benign neoplasm of pituitary gland], smoldering myeloma [Multiple myeloma not having achieved remission], smoldering myeloma [Multiple myeloma])","List(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0)","List(0, 9, 9, 9, 9, 9, 75, 12, 0, 10, 11, 10, 12, 9, 0)","List(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",mesothelioma,0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Mesothelioma,Oncological,C45,0.9986,"List(C45, C450, C459, C452, C457, C451, G731, C439, D165, C717, C649, C710, D352, C9000, C900)","List(mesothelioma [Mesothelioma], mesothelioma of pleura [Mesothelioma of pleura], mesothelioma, unspecified [Mesothelioma, unspecified], mesothelioma of pericardium [Mesothelioma of pericardium], mesothelioma of other sites [Mesothelioma of other sites], mesothelioma of mesentery [Mesothelioma of peritoneum], cancer, mesothelioma [Lambert-Eaton syndrome in neoplastic disease], amelanotic melanoma [Malignant melanoma of skin, unspecified], ameloblastoma of mandible [Benign neoplasm of lower jaw bone], glioma of brainstem [Malignant neoplasm of brain stem], nephroblastoma [Malignant neoplasm of unspecified kidney, except renal pelvis], glioblastoma multiforme, cerebrum [Malignant neoplasm of cerebrum, except lobes and ventricles], pituitary microadenoma [Benign neoplasm of pituitary gland], smoldering myeloma [Multiple myeloma not having achieved remission], smoldering myeloma [Multiple myeloma])","List(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0)","List(0, 9, 9, 9, 9, 9, 75, 12, 0, 10, 11, 10, 12, 9, 0)","List(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",mesothelioma,0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Mesothelioma,Oncological,C45,0.9986,"List(C45, C450, C459, C452, C457, C451, G731, C439, D165, C717, C649, C710, D352, C9000, C900)","List(mesothelioma [Mesothelioma], mesothelioma of pleura [Mesothelioma of pleura], mesothelioma, unspecified [Mesothelioma, unspecified], mesothelioma of pericardium [Mesothelioma of pericardium], mesothelioma of other sites [Mesothelioma of other sites], mesothelioma of mesentery [Mesothelioma of peritoneum], cancer, mesothelioma [Lambert-Eaton syndrome in neoplastic disease], amelanotic melanoma [Malignant melanoma of skin, unspecified], ameloblastoma of mandible [Benign neoplasm of lower jaw bone], glioma of brainstem [Malignant neoplasm of brain stem], nephroblastoma [Malignant neoplasm of unspecified kidney, except renal pelvis], glioblastoma multiforme, cerebrum [Malignant neoplasm of cerebrum, except lobes and ventricles], pituitary microadenoma [Benign neoplasm of pituitary gland], smoldering myeloma [Multiple myeloma not having achieved remission], smoldering myeloma [Multiple myeloma])","List(0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0)","List(0, 9, 9, 9, 9, 9, 75, 12, 0, 10, 11, 10, 12, 9, 0)","List(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",mesothelioma,0
dbfs:/FileStore/HLS/nlp/data/mt_on

In [0]:
%sql
select entity, count('*') from icd10HccView
group by 1
order by 2

entity,count(*)
Treatment,90
Oncological,439
Symptom,968


## 1.1. Get general information for staff management, reporting, & planning.

Let's take a look at the distribution of mapped codes

In [0]:
display(
  best_icd_mapped_df
  .select('onc_code_desc')
  .filter("onc_code_desc!='-'")
  .groupBy('onc_code_desc')
  .count()
  .orderBy('count')
)

onc_code_desc,count
Other disorders of the nervous system,3
Aplastic and other anemias and other bone marrow failure syndromes,3
In situ neoplasms,4
Metabolic disorders,5
Persons encountering health services for examinations,6
Malignant neoplasms of digestive organs,27
Malignant neoplasms of female genital organs,41
Melanoma and other malignant neoplasms of skin,51
"Malignant neoplasms of ill-defined, other secondary and unspecified sites",53
"Malignant neoplasms of lymphoid, hematopoietic and related tissue",83


we can also visualize the results as a countplot to see the number of each parent categories

In [0]:
import plotly.graph_objects as go

_ps=best_icd_mapped_pdf['onc_code_desc'].value_counts()
data=_ps[_ps.index!='-']

fig = go.Figure(go.Bar(
            x=data.values,
            y=data.index,
            orientation='h'))
fig.show()

## 1.2. Reimbursement-ready data with billable codes
In the previous notebook, using an icd10 oncology mapping dictionary, we created a dataset of coded conditions that are all billable. To assess the quality of the mapping, we can look at the distribution of 
the nearest billable codes

In [0]:
import plotly.express as px
import pandas as pf
_ps=best_icd_mapped_pdf['nearest_billable_code_pos'].value_counts()
data=_ps[_ps!='-']
data_pdf=pd.DataFrame({"count":data.values,"Index of Nearest Billable Codes":data.index})

fig = px.bar(data_pdf, x='Index of Nearest Billable Codes', y='count')
fig.show()

## 1.3. See which indications have the highest average risk factor
In our pipeline we used `sbiobertresolve_icd10cm_augmented_billable_hcc` as a sentence resolver, in which the model return HCC codes. We can look at the distribution risk factors for each entity.
Note that since each category has a different number of corresponding data points, to get a full picture of the distribution of risk factors for each condition, we use box plots.

In [0]:
import plotly.express as px

df = best_icd_mapped_pdf[best_icd_mapped_pdf.onc_code_desc!='-'].dropna()
fig = px.box(df, y="onc_code_desc", x="corresponding_hcc_score", hover_data=df.columns)
fig.show()

As we can see, some categories, even with fewer cases, have higher risk factor.

## 1.4. Analyze Oncological Entities
We can find the most frequent oncological entities.

In [0]:
onc_df = (
  icd10_hcc_df
  .filter("entity == 'Oncological'")
  .select("path","final_chunk","entity","icd10_code","icd_codes_names","icd_code_billable")
 )
onc_pdf=onc_df.toPandas()
onc_pdf.head(10)

,path,final_chunk,entity,icd10_code,icd_codes_names,icd_code_billable
0,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,metastatic lesions,Oncological,N40,localized bph (benign prostatic hyperplasia),0
1,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,thyroid carcinoma,Oncological,C73,medullary thyroid carcinoma,1
2,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,vocal cord,Oncological,J383,vocal cord cyst,1
3,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,papillary carcinoma,Oncological,D059,papillary carcinoma in situ of breast,0
4,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,cancer,Oncological,C801,cancer,1
5,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,Breast Cancer,Oncological,C50919,breast cancer,1
6,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,breast cancer,Oncological,C50919,breast cancer,1
7,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,tumor markers,Oncological,R97,abnormal tumor markers,0
8,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,Metastatic breast cancer,Oncological,C7981,cancer metastatic to right breast,1
9,dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_onco...,breast cancer,Oncological,C50919,breast cancer,1


In [0]:
import plotly.express as px

_ps=onc_pdf['icd_codes_names'].value_counts()
data=_ps[_ps.index!='-']
data_pdf=pd.DataFrame({"count":data.values,'icd code names':data.index})
data_pdf=data_pdf[data_pdf['count']>5]
fig = px.bar(data_pdf, y='icd code names', x='count',orientation='h')
fig.show()

### Report Counts by ICD10CM Code Names
Each bar shows count of reports contain the cancer entities.

In [0]:
display(
  onc_df.select('icd_codes_names','path')
  .dropDuplicates()
  .groupBy('icd_codes_names')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
)

icd_codes_names,count
carcinoma,10
cancer,10
carotid body tumor,8
breast cancer,6
breast cyst,6
nephrotic syndrome w focal and segmental glomerular lesions,5
basal cell carcinoma of back,4
lymphoma,4
inclusion cyst,4
cancer of the lung,4


### Most common symptoms
 We can find the most common symptoms counting the unique symptoms in documents.

In [0]:
display(
  icd10_hcc_df
  .filter("lower(entity)='symptom'")
  .selectExpr('path','icd_codes_names as symptom')
  .dropDuplicates()
  .groupBy('symptom')
  .count()
  .orderBy(F.desc('count'))
  .limit(30)
)

symptom,count
edema,18
chest mass,17
distress,16
vesicular murmur,15
symptom occurs at night (finding),14
pain,13
hepatosplenomegaly,13
lymphadenopathy,12
amblyopic,11
nausea,11


### Extract most frequent oncological diseases and symptoms based on documents

Here, we will count the number documents for each symptom-disease pair. To do this, first we filter high confidence entities and then create a pivot table.

In [0]:
entity_symptom_df = (
  icd10_hcc_df
  .select('path','entity','icd_codes_names')
  .filter("lower(entity) in ('symptom','oncological') AND confidence > 0.30")
  .dropDuplicates()
)
display(entity_symptom_df)

path,entity,icd_codes_names
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_46.txt,Symptom,distress
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,difficulty using urine bottle (finding)
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,syncope
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,Symptom,lump on neck
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,Symptom,nipple discharge
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_37.txt,Oncological,hodgkin lymphoma
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_46.txt,Oncological,mantle cell lymphoma
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,Oncological,breast cancer
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,Symptom,congestion of nose
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_7.txt,Oncological,follicular non-hodgkin's lymphoma


In [0]:
condition_symptom_df = (
  entity_symptom_df.groupBy('path').pivot("entity").agg(F.collect_list("icd_codes_names"))
  .withColumnRenamed('Oncological','Condition')
  .withColumn('Conditions',F.explode('Condition'))
  .withColumn('Symptoms',F.explode('Symptom'))
  .drop('Condition','Symptom')
  .dropna()
  .fillna(0)
)
display(condition_symptom_df)

path,Conditions,Symptoms
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,erythema
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,dysphagia
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,numbness of skin
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,swollen legs
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,blood in stool
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,dyspnea
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,difficulty in voiding
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,constipation
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,hepatosplenomegaly
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_15.txt,cancer,feeling tired


In [0]:
conditions_symptoms_count_df=condition_symptom_df.groupBy('Conditions').pivot("Symptoms").count().fillna(0)
conditions_symptoms_count_pdf=conditions_symptoms_count_df.toPandas()
conditions_symptoms_count_pdf.index=conditions_symptoms_count_pdf['Conditions']
conditions_symptoms_count_pdf=conditions_symptoms_count_pdf.drop('Conditions',axis=1)

In [0]:
selected_rows=conditions_symptoms_count_pdf.index[conditions_symptoms_count_pdf.sum(axis=1)>10]
selected_columns=conditions_symptoms_count_pdf.columns[conditions_symptoms_count_pdf.sum(axis=0)>10]

In [0]:
data_pdf=conditions_symptoms_count_pdf.loc[selected_rows,selected_columns]

Now let's visualize the heatmap of the co-occurence of conditions and symptoms. We can directly look at the counts of symptoms by condition

In [0]:
import plotly.express as px
def plot_heatmap(data,color='occurence'):
  fig = px.imshow(data,labels=dict(x="Condition", y="Symptom", color=color),y=list(data.index),x=list(data.columns))
  fig.update_layout(
    autosize=False,
    width=1100,
    height=1100,
  )
  fig.update_xaxes(side="top")
  return(fig)

In [0]:
fg=plot_heatmap(data_pdf)
fg.show()

As we see, this heatmap does not take the expected frequency of a given symptom into account. In order to reflect any correlation between the symptom in question and a given condition, we need to normalize the counts. 
To do so, we use `MinMaxScaler` to scale the values.

In [0]:
from sklearn.preprocessing import MinMaxScaler
normalized_data=MinMaxScaler().fit(data_pdf).transform(data_pdf)

In [0]:
norm_data_pdf=pd.DataFrame(normalized_data,index=data_pdf.index,columns=data_pdf.columns)
plot_heatmap(norm_data_pdf,'normalized occurence')

As we can see, now the symptoms that were not appeared to be enriched show high correlation with corresponding conditions.

# 2. Get Drug codes from the notes

## Analyze drug usage patterns for inventory management and reporting

We are checking how many times any drug are encountered in the documents.

In [0]:
rxnorm_res_df=spark.read.load(f'{delta_path}/gold/rxnorm-res-cleaned')

In [0]:
display(
  rxnorm_res_df
  .filter('confidence > 0.8')
  .groupBy('drugs')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
)

drugs,count
iron,5
Aromasin,3
Taxol,2
calcium,2
Saline Spray,2
Coreg,1
Aciphex,1
Dyazide,1
Synthroid,1
technetium Tc-99m sulfur colloid,1


# 3. Get Timeline Using RE Models

## Find the problems occured after treatments 

We are filtering the dataframe to select rows with following conditions to see problems occured after treatments.
* `relation =='AFTER'`
* `entity1=='TREATMENT'`
* `entity2=='PROBLEM'`

In [0]:
temporal_re_df=spark.read.load(f"{delta_path}/silver/temporal-re")

In [0]:
display(temporal_re_df)

path,relation,entity1,chunk1,entity2,chunk2,confidence
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,BEFORE,OCCURRENCE,Discharge,PROBLEM,Mesothelioma,0.99999833
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,PROBLEM,pleural effusion,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,PROBLEM,atrial fibrillation,0.99999607
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,Mesothelioma,PROBLEM,anemia,0.9996013
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,atrial fibrillation,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,anemia,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,ascites,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,pleural effusion,PROBLEM,esophageal reflux,0.94568694
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,atrial fibrillation,PROBLEM,anemia,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,OVERLAP,PROBLEM,atrial fibrillation,PROBLEM,ascites,1.0


In [0]:
display(
  temporal_re_df
  .where("relation == 'AFTER' AND entity1=='TREATMENT' AND entity2 == 'PROBLEM'")
  .filter('confidence > 0.8')
  .orderBy(F.desc('confidence'))
)

path,relation,entity1,chunk1,entity2,chunk2,confidence
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_21.txt,AFTER,TREATMENT,epinephrine,PROBLEM,a transverse incision,0.9999982
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_21.txt,AFTER,TREATMENT,Xylocaine,PROBLEM,a transverse incision,0.9999807
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_40.txt,AFTER,TREATMENT,Implant,PROBLEM,ruptured,0.9999635
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_14.txt,AFTER,TREATMENT,this procedure,PROBLEM,complications,0.97644377
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_20.txt,AFTER,TREATMENT,intravenous heparin,PROBLEM,hereditary hypercoagulable state,0.9599375
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_37.txt,AFTER,TREATMENT,hemostatic,PROBLEM,a skin stab inferior,0.95746464


# 4. Analyze the Relations Between Body Parts and Procedures

In the extraction notebook, we created a relation extration model to identify relationships between body parts and problem entities by using pretrained **RelationExtractionModel** `re_bodypart_problem`. Now let's load the data and take a look at the relationship between bodypart and procedures. By filtering the dataframe to select rows satisfying `entity1 != entity2` we can see the relations between different entities and see the procedures applied to internal organs

In [0]:
bodypart_re_df=spark.read.load(f'{delta_path}/silver/bodypart-relationships')

In [0]:
display(
  bodypart_re_df
  .where('entity1!=entity2')
  .drop_duplicates()
  )

path,relation,entity1,chunk1,entity2,chunk2,confidence
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,IVC,Procedure,placement of a vena caval filter,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,1,Procedure,lumpectomy,Internal_organ_or_component,axillary node,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_35.txt,0,Internal_organ_or_component,prostate,Procedure,ultrasound-guided I-125 seed implantation,0.51350594
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,inferior vena cava,Procedure,placement of a vena caval filter,1.0
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,inferior vena cava,Procedure,mechanical and pharmacologic thrombolysis,0.9999087
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,1,Internal_organ_or_component,inferior vena cava,Procedure,balloon angioplasty,0.98969346
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_8.txt,0,Procedure,bone marrow biopsy,Internal_organ_or_component,cellular marrow,0.7271675
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_45.txt,0,Procedure,lymph node injection,Internal_organ_or_component,sentinel lymph node,0.9483816
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_33.txt,0,Internal_organ_or_component,nerve,Procedure,thyroidectomy,0.99999917
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,0,Internal_organ_or_component,iliac vein,Procedure,placement of a vena caval filter,1.0


# 5. Get Procedure codes from notes

We will created dataset for procedure codes, using `jsl_ner_wip_greedy_clinical` NER mdodle and set NerConverter's WhiteList `['Procedure']` in order to get only drug entities. Let's take a look at this table:

In [0]:
cpt_df=spark.read.load(f'{delta_path}/silver/cpt')

In [0]:
display(cpt_df)

path,chunks,entity,cpt_code,confidence,all_codes,resolutions,cpt
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,decortication of the lung,Procedure,32225,0.3863,"List(32225, 32651, 39545, 36823, 32215, 33025, 1005980, 31320, 1005815, 32110, 32652, 32220, 32310, 60522, 69424, 31645, 32520)","List(Partial decortication of lung, Partial decortication of lung, Imbrication of diaphragm, Decannulation, Obliteration of pleural cavity, Decompression of pericardium, Decortication, pulmonary (separate procedure), Incision of larynx, Excision Procedures on the Larynx, Repair of lung laceration (procedure), Total decortication of lung, Total decortication of lung, Total decortication of lung, Excision of tissue of mediastinum, Removal of ventilation tube from middle ear, Tracheobronchial suctioning, Resection of lung; with resection of chest wall (Deprecated))",Partial decortication of lung
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,pleural biopsy,Procedure,32098,0.9951,"List(32098, 32095, 32609, 32604, 32400, 32491, 1007169, 37200, 75970, 32096, 32607, 32097, 32608, 10021, 32405, 32602, 32668)","List(Pleural biopsy, Open pleural biopsy, Thoracoscopy; with biopsy(ies) of pleura, Pericardial biopsy, Biopsy, pleura, percutaneous needle, Pleural procedure, Biopsy, Biopsy, Biopsy, Biopsy of lung (procedure), Biopsy of lung (procedure), Biopsy of lung (procedure), Biopsy of lung (procedure), Aspiration biopsy, Percutaneous needle biopsy lung, Thoracoscopic biopsy of lung, Pleural endoscopy)",Pleural biopsy
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,thoracentesis,Procedure,32555,0.4992,"List(32555, 32421, 32036, 1005962, 35820, 32668, 32657, 32660, 32667, 32671, 1020900, 32653, 32654, 32661, 32673, 32658, 32651)","List(Thoracentesis, Thoracentesis, Thoracostomy, Thoracostomy, Thoracostomy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy, Thoracoscopy)",Thoracentesis
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Port-A-Cath placement,Procedure,36593,0.0751,"List(36593, 75970, 49325, 20555, 62318, 75989, 62160, 64446, 55875, 49419, 41019, 19296, 61770, 19297, 64416, 64448, 55920, 0169T, 19298, 64449, 62319, 93564)","List(Catheter procedure (procedure), Catheter procedure (procedure), Catheter procedure (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Catheterization (procedure), Cardiac catheterization (procedure), Cardiac catheterization (procedure))",Catheter procedure (procedure)
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,pericardectomy,Procedure,33031,0.2498,"List(33031, 33025, 33030, 32660, 33020, 64746, 49250, 27566, 27350, 27424, 27524)","List(Pericardectomy, Pericardectomy, Pericardectomy, Pericardectomy, Pericardotomy, Phrenicectomy, Omphalectomy, Patellectomy, Patellectomy, Patellectomy, Patellectomy)",Pericardectomy
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Pericardectomy,Procedure,32660,0.2466,"List(32660, 33031, 33025, 33030, 33020, 64746, 27566, 27350, 27424, 27524)","List(Pericardectomy, Pericardectomy, Pericardectomy, Pericardectomy, Pericardotomy, Phrenicectomy, Patellectomy, Patellectomy, Patellectomy, Patellectomy)",Pericardectomy
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_0.txt,Cholecystectomy,Procedure,47145,0.1599,"List(47145, 47143, 47144, 1014153, 47600, 47605, 47562, 47620, 68520, 47490, 47480, 21070, 48000, 42335, 42340)","List(Cholecystectomy, Cholecystectomy, Cholecystectomy, Cholecystectomy, Cholecystectomy, Ch

we can the see most common procedures being performed and count the number of each procedures and plot it.

In [0]:
#top 20
display(
  cpt_df
  .groupBy('cpt')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
)

cpt,count
Biopsy,17
Bone marrow aspiration,14
Colonoscopy,11
Capsulectomy or capsulotomy,8
Pericardectomy,6
Endoscopic biopsy,6
Thoracentesis,5
Excision of chemodectoma,5
Decannulation,5
Incision AND drainage,4


# 6. Get Assertion Status of Cancer Entities

Using the assertion status dataset we can find the number of family members of cancer patients with cancer or symptoms, and we can fruther check if the symptom is absent or present.

In [0]:
assertion_df=spark.read.load(f'{delta_path}/silver/assertion').drop_duplicates()

In [0]:
display(assertion_df)

path,chunk,entity,assertion
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_34.txt,edema,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,"focal motor, sensory or other neurological symptoms",Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,Newly diagnosed high-risk acute lymphoblastic leukemia,Oncological,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_40.txt,clubbing,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,narrowing,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,decreased appetite,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_7.txt,follicular non,Cancer,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,weight loss,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,chest pains,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_43.txt,shortness of breath,Symptom,absent


In [0]:
n_associated_with_someone_else = assertion_df.where("assertion=='associated_with_someone_else'").count()
print(f"Number of family members have cancer or symptoms: {n_associated_with_someone_else} ")

Number of family members have cancer or symptoms: 35 


In [0]:
display(assertion_df)

path,chunk,entity,assertion
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_34.txt,edema,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,"focal motor, sensory or other neurological symptoms",Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_39.txt,Newly diagnosed high-risk acute lymphoblastic leukemia,Oncological,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_40.txt,clubbing,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,narrowing,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,decreased appetite,Symptom,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_7.txt,follicular non,Cancer,present
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_9.txt,weight loss,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_6.txt,chest pains,Symptom,absent
dbfs:/FileStore/HLS/nlp/data/mt_onc_50/mt_oncology_43.txt,shortness of breath,Symptom,absent


In [0]:
display(
  assertion_df
  .groupBy('assertion')
  .count()
)

assertion,count
present,570
hypothetical,13
conditional,12
possible,47
absent,496
associated_with_someone_else,35


In [0]:
assertion_symptom_df= (
  assertion_df
  .where("assertion in ('present', 'absent') AND entity=='Symptom'")
)
most_common_symptoms_df=(
  assertion_symptom_df
  .select('path','chunk')
  .groupBy('chunk')
  .count()
  .orderBy(F.desc('count'))
  .limit(20)
  )
display(most_common_symptoms_df)

chunk,count
edema,19
mass,15
murmurs,14
acute distress,12
night sweats,12
hepatosplenomegaly,12
pain,11
nausea,10
chills,10
vomiting,9


In [0]:
display(
  assertion_symptom_df
  .join(most_common_symptoms_df, on='chunk')
  .groupBy('chunk','assertion')
  .count()
  .orderBy(F.desc('count'))
  )

chunk,assertion,count
edema,absent,16
murmurs,absent,14
hepatosplenomegaly,absent,12
acute distress,absent,12
night sweats,absent,11
mass,present,10
chills,absent,10
pain,present,9
nausea,absent,9
vomiting,absent,9


## License
Copyright / License info of the notebook. Copyright [2021] the Notebook Authors.  The source in this notebook is provided subject to the [Apache 2.0 License](https://spdx.org/licenses/Apache-2.0.html).  All included or referenced third party libraries are subject to the licenses set forth below.

|Library Name|Library License|Library License URL|Library Source URL| 
| :-: | :-:| :-: | :-:|
|Pandas |BSD 3-Clause License| https://github.com/pandas-dev/pandas/blob/master/LICENSE | https://github.com/pandas-dev/pandas|
|Numpy |BSD 3-Clause License| https://github.com/numpy/numpy/blob/main/LICENSE.txt | https://github.com/numpy/numpy|
|Apache Spark |Apache License 2.0| https://github.com/apache/spark/blob/master/LICENSE | https://github.com/apache/spark/tree/master/python/pyspark|
|Plotly |MIT License| https://github.com/plotly/plotly.py/blob/master/LICENSE.txt | https://github.com/plotly/plotly.py|
|Scikit-Learn |BSD 3-Clause| https://github.com/scikit-learn/scikit-learn/blob/main/COPYING | https://github.com/scikit-learn/scikit-learn/|


|Author|
|-|
|Databricks Inc.|
|John Snow Labs Inc.|

## Disclaimers
Databricks Inc. (“Databricks”) does not dispense medical, diagnosis, or treatment advice. This Solution Accelerator (“tool”) is for informational purposes only and may not be used as a substitute for professional medical advice, treatment, or diagnosis. This tool may not be used within Databricks to process Protected Health Information (“PHI”) as defined in the Health Insurance Portability and Accountability Act of 1996, unless you have executed with Databricks a contract that allows for processing PHI, an accompanying Business Associate Agreement (BAA), and are running this notebook within a HIPAA Account.  Please note that if you run this notebook within Azure Databricks, your contract with Microsoft applies.